In [1]:
# coding: utf-8
import numpy as np
import tensorflow as tf

In [2]:
C = 10 # classes num
N = 784 # features num
fm_k = 10
learning_rate = 0.01
epochs = 2
batch_size = 128
num_steps = 2000
display_step = 100

In [3]:
# tf.zeros([X.shape.as_list()[1], C])

In [4]:
def fm(X, n_classes, n_features, k):
    W0 = tf.Variable(tf.zeros(n_classes), name='W0')
    W1 = tf.Variable(tf.zeros([n_features, n_classes]), name='W1')
    W2 = tf.Variable(tf.zeros([k, n_features, n_classes]), name='W2')
    
    term2 = None
    for i in range(0, k):
        elem = tf.square(tf.matmul(X, W2[i])) - tf.matmul(tf.pow(X, 2), tf.square(W2[i]))
        term2 = term2 + elem if term2 is not None else elem
    logits = W0 + tf.matmul(X, W1) + term2
    return logits

In [5]:
def log_loss_fn(logits, labels, l2_reg=0):
    logits_loss = tf.losses.sparse_softmax_cross_entropy(labels, logits=logits)
    return logits_loss, logits_loss, logits_loss
#     W = tf.get_default_graph().get_tensor_by_name('W:0')
#     l2_loss = l2_reg * tf.nn.l2_loss(W)
#     return logits_loss + l2_loss, logits_loss, l2_loss

In [6]:
# def hinge_loss_fn(logits, labels, l2_reg=0):
# #     logits_loss = tf.losses.hinge_loss(labels, logits=logits)
#     labels = (tf.one_hot(labels, C) - 0.5) * 2
#     logits_loss = tf.reduce_mean(tf.maximum(0.0, 1 - logits * labels))
#     W = tf.get_default_graph().get_tensor_by_name('W:0')
#     l2_loss = l2_reg * tf.nn.l2_loss(W)
#     return logits_loss + l2_loss, logits_loss, l2_loss

In [7]:
def accuracy_fn(logits, labels):
    predictions = tf.nn.softmax(logits)
    is_correct = tf.equal(tf.argmax(predictions, axis=1), labels)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    return accuracy

In [8]:
def train_input_fn(features, labels, batch_size, shuffle_size=1000):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(shuffle_size).repeat().batch(batch_size).prefetch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
x_train, y_train = mnist.train.images, mnist.train.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [10]:
class CumulateAvg:
    def __init__(self, var_list, display_step=100):
        self.var_list = var_list
        self.reset()
        self.display_step = display_step
        self.step = 0
        
    def reset(self):
        self.var_dict = {k: 0 for k in self.var_list}
        
    def avg_var(self):
        if self.step > 0:
            self.var_dict = {k: v / self.display_step for k, v in self.var_dict.items()}
        
    def cumulate(self, var_values):
        for i, v in enumerate(var_values):
            self.var_dict[self.var_list[i]] += v
        if self.step == 0 or (self.step + 1) % self.display_step == 0:
            self.avg_var()
            print_str = 'step: %d' % (self.step + 1)
            print_str += ''.join([', %s: %.4f' % (k, self.var_dict[k]) for k in self.var_list])
            print(print_str)
        self.step += 1

In [11]:
with tf.Graph().as_default() as g:
    with tf.Session().as_default() as sess:
        dataset = train_input_fn(x_train, y_train, batch_size)
        X, y = dataset
        X = tf.cast(X, tf.float32)
        y = tf.cast(y, tf.int64)
        
        logits = fm(X, C, N, fm_k)
        
        loss, logits_loss, l2_loss = log_loss_fn(logits, y, l2_reg=0.05)
#         loss, logits_loss, l2_loss = hinge_loss_fn(logits, y, l2_reg=0.05)

        accuracy = accuracy_fn(logits, y)
        train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        sess.run(tf.global_variables_initializer())

        var_list = ['loss', 'accuracy', 'logits_loss', 'l2_loss']
        cumavg_obj = CumulateAvg(var_list, 100)
        for step in range(num_steps):
            res_list = sess.run([train_op, *[eval(x) for x in var_list]])
            cumavg_obj.cumulate(res_list[1:])

step: 1, loss: 2.3026, accuracy: 0.0859, logits_loss: 2.3026, l2_loss: 2.3026
step: 100, loss: 1.8882, accuracy: 0.6845, logits_loss: 1.8882, l2_loss: 1.8882
step: 200, loss: 1.3751, accuracy: 0.7983, logits_loss: 1.3751, l2_loss: 1.3751
step: 300, loss: 1.1245, accuracy: 0.8228, logits_loss: 1.1245, l2_loss: 1.1245
step: 400, loss: 0.9479, accuracy: 0.8364, logits_loss: 0.9479, l2_loss: 0.9479
step: 500, loss: 0.8232, accuracy: 0.8538, logits_loss: 0.8232, l2_loss: 0.8232
step: 600, loss: 0.7824, accuracy: 0.8481, logits_loss: 0.7824, l2_loss: 0.7824
step: 700, loss: 0.7494, accuracy: 0.8554, logits_loss: 0.7494, l2_loss: 0.7494
step: 800, loss: 0.7023, accuracy: 0.8539, logits_loss: 0.7023, l2_loss: 0.7023
step: 900, loss: 0.6158, accuracy: 0.8808, logits_loss: 0.6158, l2_loss: 0.6158
step: 1000, loss: 0.6411, accuracy: 0.8608, logits_loss: 0.6411, l2_loss: 0.6411
step: 1100, loss: 0.6222, accuracy: 0.8682, logits_loss: 0.6222, l2_loss: 0.6222
step: 1200, loss: 0.5986, accuracy: 0.86